In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
!pip install torchkge

In [2]:
%%capture
!conda install -y gdown

In [3]:
!gdown --id 1-383Q6iPXXNr047ZCw7g6QjkDcbA8-B4
!gdown --id 13YNwGftMtzdb0dL2OC7n3gsGsCSCHFpq
!gdown --id 1-5-2u3gGWpHcjOeSBDsF5okGbAOzo6dq

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-383Q6iPXXNr047ZCw7g6QjkDcbA8-B4
To: /kaggle/working/test_df.csv
100%|█████████████████████████████████████████| 156M/156M [00:01<00:00, 129MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13YNwGftMtzdb0dL2OC7n3gsGsCSCHFpq
To: /kaggle/working/train_df.csv
100%|█████████████████████████████████████████| 271M/271M [00:02<00:00, 130MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don

In [4]:
# basics
import pandas as pd
import numpy as np
from os.path import join
from os import makedirs, remove
from os.path import exists
from tqdm.autonotebook import tqdm
import pickle
import gc

# torchkge related
import torch
from torch import cuda
from torch.optim import Adam
from torchkge.models import TransRModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
from torchkge.data_structures import KnowledgeGraph
from torchkge.utils import get_data_home
from torchkge.utils import Trainer
from torchkge.evaluation import LinkPredictionEvaluator

# Model path for colab
# MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/models'
# Model path for Kaggle
MODEL_PATH = '/kaggle/working/'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [ ]:
# set up cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [5]:
gc.collect()

24

In [6]:
def load_kgs_from_df(data_home=None):
#     data_home = '/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/splits'
    data_home = '/kaggle/working'
    df1 = pd.read_csv(data_home + '/train_df.csv')
    df2 = pd.read_csv(data_home + '/val_df.csv')
    df3 = pd.read_csv(data_home + '/test_df.csv')
    df = pd.concat([df1, df2, df3])
    kg = KnowledgeGraph(df)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

In [7]:
# RUN ONCE
kg_train, kg_val, kg_test = load_kgs_from_df()

In [41]:
gc.collect()

102

In [26]:
# Define some hyper-parameters for training
emb_dim = 100
lr = 0.0004
margin = 0.5
n_epochs = 100
b_size = 9143

In [27]:
model = TransRModel(emb_dim, emb_dim, kg_train.n_ent, kg_train.n_rel)
criterion = MarginLoss(margin)
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
trainer = Trainer(model, criterion, kg_train, n_epochs, b_size,
                  optimizer=optimizer, sampling_type='bern', use_cuda='all')

In [28]:
trainer.run()

  0%|          | 0/100 [00:00<?, ?epoch/s]

In [57]:
evaluator = LinkPredictionEvaluator(model, kg_test)
evaluator.evaluate(b_size=100, verbose=True)
evaluator.print_results()

Link prediction evaluation:   0%|          | 0/41399 [00:00<?, ?batch/s]

AttributeError: 'TransRModel' object has no attribute 'emb_dim'